In [1]:
import qcodes as qc
station = qc.Station(config_file='config/station.yaml')

In [2]:
dac1 = station.load_mock_dac1()
dac2 = station.load_mock_dac2()
mote2general = station.load_mote2_general()

2025-09-07 15:04:22,821 - servers.jupyter_qcodes.tests.data_loader - INFO - Loading data from /Users/caijiaqi/Documents/GitHub/instrMCP/servers/jupyter_qcodes/tests/data_file/MoTe2QAHE/processed_data.mat
2025-09-07 15:04:22,846 - servers.jupyter_qcodes.tests.data_loader - INFO - Data validation passed: Vtg(220), Vbg(900), lockin arrays(900x220)
2025-09-07 15:04:22,846 - servers.jupyter_qcodes.tests.data_loader - INFO - Data loaded and validated successfully
2025-09-07 15:04:22,847 - servers.jupyter_qcodes.tests.test_instr.general_test_instrument - INFO - Data loaded from /Users/caijiaqi/Documents/GitHub/instrMCP/servers/jupyter_qcodes/tests/data_file/MoTe2QAHE/processed_data.mat
2025-09-07 15:04:22,848 - servers.jupyter_qcodes.tests.test_instr.general_test_instrument - INFO - General test instrument 'mote2_general' initialized from ${instrMCP_PATH}/servers/jupyter_qcodes/tests/data_file/MoTe2QAHE/MoTe2Para.json (expanded: /Users/caijiaqi/Documents/GitHub/instrMCP/servers/jupyter_qcodes

In [3]:
%load_ext servers.jupyter_qcodes.jupyter_mcp_extension

2025-09-07 15:04:24,857 - servers.jupyter_qcodes.jupyter_mcp_extension - INFO - Loading Jupyter QCoDeS MCP extension...
2025-09-07 15:04:24,859 - servers.jupyter_qcodes.active_cell_bridge - INFO - Registered comm target 'mcp:active_cell'
2025-09-07 15:04:24,860 - servers.jupyter_qcodes.tools - INFO - QCoDesReadOnlyTools initialized
2025-09-07 15:04:24,872 - servers.jupyter_qcodes.mcp_server - INFO - Jupyter MCP Server initialized on 127.0.0.1:8123 in safe mode
2025-09-07 15:04:24,873 - servers.jupyter_qcodes.jupyter_mcp_extension - INFO - Jupyter QCoDeS MCP extension loaded successfully


✅ QCoDeS MCP extension loaded
📋 Use %mcp_status to check server status
🚀 Use %mcp_start to start the server


In [6]:
%mcp_status

⚠️ MCP Server Status:
   Mode: unsafe
   Running: ❌
   Host: 127.0.0.1:8123
   Task: ❌ Inactive
   Unsafe tools: execute_editing_cell
   Available: %mcp_start


In [5]:
%mcp_unsafe

2025-09-07 15:04:31,361 - servers.jupyter_qcodes.mcp_server - INFO - MCP server mode changed from safe to unsafe


⚠️  MCP server switched to unsafe mode
⚠️  UNSAFE MODE: execute_editing_cell tool will be available
✅ Mode change will take effect when server starts


In [7]:
%mcp_start

2025-09-07 15:04:36,432 - servers.jupyter_qcodes.mcp_server - INFO - Starting Jupyter MCP server on 127.0.0.1:8123


🚀 Starting MCP server...
🚀 QCoDeS MCP Server running on http://127.0.0.1:8123
🔑 Access token: 4aPCe__sq2vBtpSVhY_3fTm72Immq3PUO6AN8qyJG_Y


/Users/caijiaqi/Documents/GitHub/instrMCP/venv/lib/python3.10/site-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09
/Users/caijiaqi/Documents/GitHub/instrMCP/venv/lib/python3.10/site-packages/uvicorn/protocols/websockets/websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol
INFO:     Started server process [81011]
INFO:     Waiting for application startup.
2025-09-07 15:04:36,484 - mcp.server.streamable_http_manager - INFO - StreamableHTTP session manager started
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8123 (Press CTRL+C to quit)


✅ MCP server started in ⚠️ unsafe mode


2025-09-07 15:04:46,141 - mcp.server.streamable_http_manager - INFO - Created new transport with session ID: 7bd50fce74574d6fa0fd0451cabf5a29


...

2025-09-07 15:04:47,482 - mcp.server.lowlevel.server - INFO - Processing request of type ListToolsRequest
2025-09-07 15:06:30,873 - mcp.server.streamable_http - INFO - Terminating session: 7bd50fce74574d6fa0fd0451cabf5a29


INFO:     127.0.0.1:58790 - "DELETE /mcp HTTP/1.1" 200 OK


2025-09-07 15:06:58,375 - mcp.server.streamable_http_manager - INFO - Created new transport with session ID: 3f12bbe0b54242f09314a85a492ab7ea


INFO:     127.0.0.1:58847 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:58847 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:58849 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:58847 - "POST /mcp HTTP/1.1" 200 OK


2025-09-07 15:07:00,570 - mcp.server.lowlevel.server - INFO - Processing request of type ListToolsRequest


In [ ]:
dac1.print_readable_snapshot()